In [10]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics

In [13]:
path ="C:/Users/moumouni/Desktop/OC_P5_ml/Stack_questions_cleaned.csv"
data = pd.read_csv(path,sep=';')
from ast import literal_eval
for col in ['Title', 'Body', 'Tags', 'text_comb']:
     data[col] = data[col].apply(literal_eval) 
     
print(data.head())

# transform target tags
mlb = MultiLabelBinarizer()
mlb.fit(data.Tags)
y_mlb = mlb.transform(data.Tags)
print("Number of Tags:", len(mlb.classes_))
#
# data split
X_train, X_test, y_train, y_test = train_test_split(data.text_comb,\
                    y_mlb, test_size=0.25, random_state=42)


                                             Title  \
0                   [comma, array, print, bracket]   
1                          [directory, c, listing]   
2                        [return, wcf, datatables]   
3                        [error, visible, element]   
4  [resolution, opencv, camera, increase, capture]   

                                                Body                    Tags  \
0  [fit, console, game, original, format, comma, ...  [array, android, java]   
1  [scan, c, file, folder, platform, cross, direc...               [c, file]   
2  [table, underlie, timeouts, data, connection, ...           [c#, service]   
3  [demo, count, tesstcase, error, syntaxerror, e...            [javascript]   
4  [webcam, resolution, opencv, quickcam, c++, im...              [c, image]   

                                           text_comb  
0  [fit, console, game, original, format, comma, ...  
1  [scan, c, file, folder, platform, cross, direc...  
2  [table, underlie, timeouts

In [15]:
# define model
final_model = Pipeline([('tfidf', TfidfVectorizer(analyzer="word",
                             max_df=.95,
                             min_df= 5,
                             tokenizer=None,
                             preprocessor=' '.join,
                             stop_words=None,
                             lowercase=False)),
                        ('lr', OneVsRestClassifier(LogisticRegression()))])

#fit model

final_model.fit(X_train,y_train)

#prediction and score
y_score = final_model.predict_proba(X_test)
print('Score ROC_AUC:', metrics.roc_auc_score(y_test, y_score,multi_class="ovr"))

Score ROC_AUC: 0.9692983542147782


In [16]:
from pickle import dump
dump(final_model, open("final_model.pkl","wb"))